In [1]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os 
import psycopg

# load .env contstants
load_dotenv()

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = {
    "host": os.getenv("DB_DESTINATION_HOST"),
    "port": os.getenv("DB_DESTINATION_PORT"),
    "dbname": os.getenv("DB_DESTINATION_NAME"),
    "user": os.getenv("DB_DESTINATION_USER"),
    "password": os.getenv("DB_DESTINATION_PASSWORD"),
}

# Create a connection string
connection_string = (
    f"postgresql://{postgres_credentials['user']}:{postgres_credentials['password']}"
    f"@{postgres_credentials['host']}:{postgres_credentials['port']}/{postgres_credentials['dbname']}"
)

# Create a SQLAlchemy engine
engine = create_engine(connection_string)


def load(TABLE_NAME):
    connection.update(postgres_credentials)
    with psycopg.connect(**connection) as conn:
        with conn.cursor() as cur:
            cur.execute(f"SELECT * FROM {TABLE_NAME}")
            data = cur.fetchall()
            columns = [col[0] for col in cur.description]
    df = pd.DataFrame(data, columns=columns)

    return df

In [24]:
from sklearn.preprocessing import LabelEncoder

events = load("events")

item_encoder = LabelEncoder()
user_encoder = LabelEncoder()
events["timestamp"] = pd.to_datetime(events['timestamp'], unit='ms')

item_encoder.fit(events["itemid"])
user_encoder.fit(events["visitorid"])

events.loc[:, 'visitorid'] = user_encoder.transform(events['visitorid'])
events.loc[:, 'itemid'] = item_encoder.fit_transform(events['itemid'])

In [25]:
train_test_global_time_split_date = pd.to_datetime("2015-09-18")

train_test_global_time_split_idx = events["timestamp"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

In [31]:
add_to_cart = events_train[events_train["event"]=="addtocart"]
view_but_no_cart = events_train[(events_train["event"]!="addtocart") & (events_train["event"]=="view")]

In [33]:
import numpy as np

In [52]:

add_to_cart = events_train[events_train["event"]=="addtocart"]
view_but_no_cart = events_train[(events_train["event"]!="addtocart") & (events_train["event"]=="view")]


k = 5 # significants factor,i.e how much add to chart is more important compared to view only (tunable)
scores = list(np.ones(view_but_no_cart.shape[0])) + list(np.ones(add_to_cart.shape[0])*k)

idx_users = list(view_but_no_cart["visitorid"].values.astype(int))+list(add_to_cart["visitorid"].values.astype(int))
idx_items = list(view_but_no_cart["itemid"].values.astype(int))+list(add_to_cart["itemid"].values.astype(int))

import scipy

user_item_matrix_train = scipy.sparse.csr_matrix((
    scores,
    idx_users, idx_items),
    dtype=np.int8)

ValueError: index pointer should start with 0

In [51]:
import scipy

user_item_matrix_train = scipy.sparse.csr_matrix((
    scores,
    idx_users, idx_items),
    dtype=np.int8)

ValueError: index pointer should start with 0

In [ ]:
import numpy as np
import scipy


user_item_matrix_train = scipy.sparse.csr_matrix((
    np.ones(events_train.shape[0]),
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)